# Escrapeo DIA

Lo haremos solo con bs4, debido a que la navegabilidad por la página con selenium es complicada

https://www.dia.es/compra-online/#

In [2]:
# Importamos librerías necesarias para escrapear

import requests as req

from bs4 import BeautifulSoup as bs

import pandas as pd

In [4]:
# Importamos el excel creado de categorias y subcategorías
df_dia = pd.read_excel(r'C:\Users\mario\OneDrive\Escritorio\Proyecto Página Web\Escrapeo-DIA\DIA_cat.xlsx')

# Eliminamos las filas que no nos sirven
df_dia = df_dia.drop(df_dia.index[[0, 1]])

# Reseteamos el indice
df_dia.reset_index(inplace=True, drop=True)

# Renombramos columnas
df_dia.columns = ['cat', 'scat', 'npg']

In [5]:
# Creamos las URls
url_cat = ['https://www.dia.es/compra-online/{}/'.format(cat) for cat in df_dia.cat.tolist()]

url_scat = []
for i, scat in enumerate(df_dia.scat.tolist()):
    url_scat.append(url_cat[i] + '{}/cf'.format(scat))

In [6]:
url_scat[:5]

['https://www.dia.es/compra-online/dulces-de-navidad/turrones/cf',
 'https://www.dia.es/compra-online/dulces-de-navidad/surtidos-navidenos/cf',
 'https://www.dia.es/compra-online/dulces-de-navidad/panettones/cf',
 'https://www.dia.es/compra-online/dulces-de-navidad/roscones-de-reyes/cf',
 'https://www.dia.es/compra-online/platos-preparados/pizzas-refrigeradas/cf']

In [7]:
# Filtramos el DataFrame con las subcategorias que tienen más de una página
df_dia_filt = df_dia[df_dia.npg > 1]

index = df_dia_filt.index.to_list()

# Cogemos las URLs de las subcategorias que tienen más de una página
url_scat_filt = [url_scat[i] for i in index]

# Sacamos una lista del numero de páginas de cada subcategoría
npg = df_dia_filt.npg.tolist()

In [8]:
print(len(url_scat_filt), ',', len(npg))

51 , 51


In [9]:
# Sacamos las URLs de las subcategorias que tienen varias páginas
new_url = []

for url, n in zip(url_scat_filt, npg):
    for i in range(1, n):
        new_url.append(url + '/cf?page={}&disp='.format(i))

In [10]:
new_url[:5]

['https://www.dia.es/compra-online/frescos/carne/cf/cf?page=1&disp=',
 'https://www.dia.es/compra-online/frescos/carne/cf/cf?page=2&disp=',
 'https://www.dia.es/compra-online/frescos/carne/cf/cf?page=3&disp=',
 'https://www.dia.es/compra-online/frescos/charcuteria-y-quesos/cf/cf?page=1&disp=',
 'https://www.dia.es/compra-online/frescos/charcuteria-y-quesos/cf/cf?page=2&disp=']

In [11]:
# Unimos en una única lista todas las URLs
urls = url_scat + new_url

In [12]:
len(urls)

291

In [13]:
## A partir de aquí comienza la extracción del HTML de todas las páginas
html = []

for url in urls:
    html.append(req.get(url).text)

In [14]:
soup = [bs(html[i], 'html.parser') for i in range(len(html))]

In [15]:
len(soup)

291

In [101]:
# Sacamos las celdas de los productos
products_grid_list = []

for i in range(len(soup)):
    products_grid_list.append(soup[i].find_all(class_ = 'product-list__item'))
    
# Desglosamos las listas
products_grid = []

for item in products_grid_list:
    products_grid += item

In [102]:
len(products_grid)

5886

In [106]:
# Sacamos los links de las imágenes de los productos
product_img = []
for i in range(len(products_grid)):
    try:
        product_img.append(products_grid[i].find(class_ = 'crispImage').attrs['src'])
    except:
        product_img.append('')

In [107]:
product_img[:10]

['https://s0.dia.es/medias/productimages/h2d/hc6/10837627568158.jpg',
 'https://s3.dia.es/medias/productimages/h96/h78/10837610528798.jpg',
 'https://s0.dia.es/medias/productimages/h15/hea/10874375405598.jpg',
 'https://s2.dia.es/medias/productimages/ha3/ha7/10839002054686.jpg',
 'https://s2.dia.es/medias/productimages/hd6/hd0/10871302291486.jpg',
 'https://s0.dia.es/medias/h9c/h3e/10351178383390.jpg',
 'https://s0.dia.es/medias/productimages/h35/h7a/10871280009246.jpg',
 'https://s1.dia.es/medias/h34/h44/10389740486686.jpg',
 'https://s1.dia.es/medias/h38/h7a/10389747793950.jpg',
 'https://s0.dia.es/medias/productimages/he5/he4/10870041772062.jpg']

In [108]:
# Sacamos los nombres de los productos
product_name = []
for i in range(len(products_grid)):
    try:
        product_name.append(products_grid[i].find(class_ = 'details').text.strip().lower())
    except:
        product_name.append('')

In [109]:
product_name[:10]

['dia dulce noel turrón imperial de cacahuete estuche 150 gr',
 'dia dulce noel turrón de crema de cacahuete estuche 150 gr',
 'nestle lote chocolate extrafino y peluche caja 260 gr (modelo aleatorio)',
 'kinder bueno mini happy moments caja 191 gr',
 'la hornada dia pandoro con chocolate bolsa 750 gr',
 'la hornada dia pandoro clásico bolsa 750 gr',
 'la hornada dia mini panettone de chocolate bolsa 100 gr',
 'dia dulce noel roscón de reyes sin relleno caja 400 gr',
 'dia dulce noel roscón de reyes relleno sabor trufa caja 550 gr',
 'casa tarradellas pizza jamón y queso envase 405 gr']

In [110]:
# Sacamos los precios de los productos
product_price = []
for i in range(len(products_grid)):
    try:
        product_price.append(products_grid[i].find(class_ = 'price').text.strip())
    except:
        product_price.append('')

In [111]:
# Limpiamos los precios
product_price = [product_price[i][:4] for i in range(len(product_price))]
product_price = [float(product_price[i].replace(',', '.')) for i in range(len(product_price))]

In [112]:
product_price[:10]

[0.99, 0.99, 7.49, 5.49, 3.99, 3.99, 0.99, 3.49, 3.99, 2.63]

In [115]:
# Vemos el número de productos que hay en cada página
n_products = []
for i in range(len(soup)):
    n_products.append(len(soup[i].find_all(class_ = 'product-list__item')))

In [116]:
# Sacamos la lista completa de los nombres de las subcategorías
scat_list = []

for scat, n in zip(products_scat, n_products):
    scat_list.append([scat]*n)

# Desglosamos las listas
scat_names = []

for item in scat_list:
    scat_names += item

In [117]:
# Vemos si el tamaño de todas las listas coincide
print(len(product_name), len(product_price), len(product_img), len(scat_names)) # Coinciden

5886 5886 5886 5886


In [120]:
# Cremos el DataFrame de los productos
df_dia_products = pd.DataFrame({'product' : product_name,
                            'price' : product_price,
                            'scat' : scat_names,
                            'img' : product_img})

In [121]:
df_dia_products.head()

,product,price,scat,img
0,dia dulce noel turrón imperial de cacahuete es...,0.99,turrones,https://s0.dia.es/medias/productimages/h2d/hc6...
1,dia dulce noel turrón de crema de cacahuete es...,0.99,turrones,https://s3.dia.es/medias/productimages/h96/h78...
2,nestle lote chocolate extrafino y peluche caja...,7.49,surtidos navideños,https://s0.dia.es/medias/productimages/h15/hea...
3,kinder bueno mini happy moments caja 191 gr,5.49,surtidos navideños,https://s2.dia.es/medias/productimages/ha3/ha7...
4,la hornada dia pandoro con chocolate bolsa 750 gr,3.99,panettones,https://s2.dia.es/medias/productimages/hd6/hd0...


In [123]:
df_dia_products.to_excel(r'C:\Users\mario\OneDrive\Escritorio\Proyecto Página Web\Escrapeo-DIA\productos_dia.xlsx', index = False)